In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import labolibrary as labo
import pickle
from sklearn.preprocessing import RobustScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [11]:

#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'      

scalers = {}
# Function to center, scale, and return a series
def scale_group(group):
    scaler = RobustScaler()
    scaled_values = scaler.fit_transform(group.values.reshape(-1, 1)).flatten()
    scalers[group.name] = scaler  # Store the scaler for this group
    return pd.Series(scaled_values, index=group.index, name=group.name)

# Function to inverse transform (de-scale) and decenter, and return a series
def inverse_scale_group(group):
    group_name = group.name
    scaler = scalers[group_name]
    inversed_centered_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    original_values = inversed_centered_values
    return pd.Series(original_values, index=group.index, name=group_name)


In [12]:

# Leer datos
#df_final = pd.read_parquet(DATOS_DIR+'FE_dataset-CARLA.parquet') 
df_final = pd.read_parquet(DATOS_DIR+'/FE_02_dataset.parquet') 
df_final.columns = df_final.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)




In [13]:
#weight = df_final.groupby(['product_id', df_final.index])['weight'].transform('mean')

df_final.groupby('product_id')['weight'].transform(scale_group)


periodo
2017-01     0.222190
2017-01     1.444379
2017-01    -0.111242
2017-01    -0.222336
2017-01    22.111242
             ...    
2019-12     0.384748
2019-12    -0.153899
2019-12     0.615252
2019-12     0.153899
2019-12     0.153899
Freq: M, Name: weight, Length: 2293481, dtype: float64

In [14]:

### Filtrar datos
df_true = df_final.loc['2019-12-01':'2020-01-01']
df_final = df_final.loc['2018-01-01':'2019-11-01']


#Filtro de no compradores
#Step 1: Ensure the index is a datetime type
df_final.index = df_final.index.to_timestamp()
# Step 2: Determine the last date and calculate the date 3 months prior
ls_date  = df_final.index.max()
three_months_prior = ls_date - pd.DateOffset(months=3)

# Step 3: Filter the dataframe to include onl+y rows within the last 3 months
last_3_months_df = df_final[df_final.index >= three_months_prior]

# Step 4: Identify the unique client_id that have purchased within this period
active_clients = last_3_months_df['customer_id'].unique()

# Step 5: Filter the original dataframe to include only these client_id
df_final = df_final[df_final['customer_id'].isin(active_clients)]

df_final.index = pd.PeriodIndex(df_final.index, freq='M')


#Filtro test
#df_final = df_final[df_final['product_id'] < 20013]

weight= df_final[['weight','product_id']]
df_final.drop(columns=['weight'], inplace=True)


In [15]:
# Correr Modelo
params={
        'boosting_type': 'gbdt',
        'objective': 'Regression',
        'metric':'rmse',
        'verbose': -1,
        #'n_jobs': -1,
        'seed': 10000079,
        #'learning_rate': 0.2,
        'bagging_fraction': 0.85,
        'bagging_freq': 1, 
        #'colsample_bytree': 0.85,
        #'colsample_bynode': 0.85,
        #'min_data_per_leaf': 25,
        #'num_leaves': 200,
        #'lambda_l1': 0.5,
        #'lambda_l2': 0.5
}

predictions_all = pd.DataFrame(columns=['tn'])
products = df_final['product_id'].unique()
tot = len(products)
nro = 0
for producto in products:
    print(f'Fitting and predicting for product_id: {producto}')
    # Filtrar los datos del producto
    df_producto = df_final[df_final['product_id'] == producto]
    weight_p= weight[weight['product_id'] == producto]
    # Prepare data for LSTM on tn_2 only
    X = df_producto[['tn_2']].values.astype('float32')
    y = df_producto['tn_2'].values.astype('float32')
    X = X.reshape((X.shape[0], 1, X.shape[1]))
    #######################################################    
    # Define LSTM model
    model = Sequential()
    model.add(LSTM(50, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy')

    # Train LSTM model
    model.fit(X, y, epochs=10, batch_size=32, verbose=1)

    # Extract features from LSTM
    features = model.predict(X)

    # Prepare data for LightGBM
    # Convert LSTM features to DataFrame
    features_df = pd.DataFrame(features, index=df_producto.index, columns=[f'lstm_feature_{i}' for i in range(features.shape[1])])
    df_producto = pd.concat([df_producto, features_df], axis=1)
    df_producto['tn_2'] = y
    
    #############################################################################  
    #### Agrupar y escalar
   
    model, average_metric = labo.train_lightgbm_model(df_producto,params,
                                                      weights=weight_p['weight'])
    print("Overall rmse metric: ", average_metric)
    # Predict values for the entire dataset using the trained models
    # Prepare last data points for prediction
    last_data_points = df_producto[df_producto.index == df_producto.index.max()].copy()
    last_data_points.drop(columns=['tn_2'], inplace=True)
    # Predict the next month's value using the trained model
    predictions = labo.predict_next_month(model, last_data_points)
    preds = predictions.groupby('product_id')['tn_2'].transform(inverse_scale_group)
    predictions['tn'] = preds
    predictions.drop(columns=['tn_2'], inplace=True)
    predictions = predictions.reset_index()
    predictions =  predictions.groupby('product_id')['tn'].sum()
    predictions.columns = ['product_id', 'tn']
    predictions_all = pd.concat([predictions_all, predictions])
    print(predictions_all[-1:])


Fitting and predicting for product_id: 20524
Epoch 1/10


d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 733us/step - loss: 0.3655
Epoch 2/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 728us/step - loss: -0.9335
Epoch 3/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -2.6864
Epoch 4/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - loss: -7.2743
Epoch 5/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 942us/step - loss: -11.6659
Epoch 6/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - loss: -12.7240
Epoch 7/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 812us/step - loss: -13.6607
Epoch 8/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - loss: -14.8626
Epoch 9/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step - loss: -18.0052
Epoch 10/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 861us/step - loss: -22.1005
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[5]	validation's rmse: 2.58991	validation's multinacional_metric: 0.00610787
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[15]	validation's r

C:\Users\Admin\AppData\Local\Temp\ipykernel_18048\376699480.py:70: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])
C:\Users\Admin\AppData\Local\Temp\ipykernel_18048\376699480.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])
d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Seq

68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 704us/step - loss: -0.8446 
Epoch 2/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 655us/step - loss: -4.2548
Epoch 3/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 672us/step - loss: -8.8828
Epoch 4/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 661us/step - loss: -13.2351
Epoch 5/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 680us/step - loss: -16.2086
Epoch 6/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 728us/step - loss: -20.1491
Epoch 7/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step - loss: -24.1015
Epoch 8/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -31.9760
Epoch 9/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 715us/step - loss: -37.2607
Epoch 10/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step - loss: -43.9490
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[13]	validation's rmse: 3.77186	validation's multinacional_metric: 0.00193266
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[19]	validati

...

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 925us/step - loss: 1.2542
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 946us/step - loss: 0.3689
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1801 
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.3490 
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: -0.9284 
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -1.2454 
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -1.9813 
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -1.2879
Epoch 9/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 950us/step - loss: -2.3130
Epoch 10/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 969us/step - loss: -2.1390
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[16]	validation's rmse: 1.49133	validation's multinacional_metric: 0.460433
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[29]	validation's rmse: 4.19095	validation's multinaci

d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6898
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 981us/step - loss: 0.5666
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4759 
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 949us/step - loss: 0.2843
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1178  
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 898us/step - loss: 0.0545
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step - loss: -0.1892
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - loss: -0.3081
Epoch 9/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.2287
Epoch 10/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 982us/step - loss: -0.9729
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	validation's rmse: 1.33755	validation's multinacional_metric: 0.56643
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[48]	validation's rmse: 5.01606	validation's multinaciona

d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.7504
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5087 
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1800 
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0392 
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 844us/step - loss: -0.6747
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.5291 
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.9516 
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -1.0295 
Epoch 9/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -1.5296 
Epoch 10/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -1.6928 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	validation's rmse: 5.87274	validation's multinacional_metric: 0.0189902
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[5]	validation's rmse: 2.84796	validation's multinacional_

In [16]:

predictions_all['tn']=predictions_all['tn'].astype('float32')
predictions_all.index.names = ['product_id']
predictions_all.to_csv(DATOS_DIR+'/pred/0022-prediccion-custom_scaled_Robust-product_id_LSTM-No_Buyer_weights.csv', index=True,header=True)
print("Overall custom metric: ", average_metric)


Overall custom metric:  0.006158239872136777
